In [ ]:
import numpy as np
from sklearn import decomposition, svm
from matplotlib import pyplot as plt

from main import get_embeddings
from torch.utils.data import DataLoader


In [ ]:
get_embeddings(40.667808, -74.063452, "2022-01-01", "2022-01-03")

In [ ]:
points_true = [  # stadium
  (51.45576189740324, -0.34101157597801884),
  (51.555997989240666, -0.2800146693353107), 
  (51.481657256405754, -0.19071106803328688),
  (53.46276505480401, -2.2911469832733213),
  (51.55492197276949, -0.10837834474336895)
]

points_false = [  # parks
  (51.509726724102165, -0.16431432775703508),
  (51.505942185188594, -0.17765388975199076),
  (51.53050581740778, -0.1531204067394265),
  (51.53733131004048, -0.038022943072186524),
]

points_test = [
  (51.538526217382234, -0.01625981008911321), # stadium
  (52.239603606778324, 21.046021829807653), # stadium
  (51.564101793728085, -0.16598854545362213), # park
  (51.55775528574248, 0.05150102398763338), # park
]

"""

nyc_tennis_courts = [
  (40.721742, -73.954551), # 7
  (40.716006, -73.936375), # 2
  (40.690549, -73.975471), # 6
  (40.689823, -73.999991), # 2
  (40.683636, -73.972244), # 2
  (40.680305, -73.928017), # 4
  (40.683585, -73.888442), # 4
  (40.683937, -73.887713), # 4
  (40.768417, -73.943442), # 6
  (40.761025, -73.894467), # 2
  (40.776046, -73.924727), # 4
  (40.775763, -73.924510), # 4
  (40.775475, -73.924287), # 4
]
nyc_not_tennis_court = [
  (40.631173, -73.990512),
  (40.617278, -73.995161),
  (40.627687, -74.016759),
  (40.628424, -74.040643),
  (40.636053, -74.039432),
  (40.652933, -74.019090),
  (40.653229, -73.992678),
  (40.653229, -73.992678),
  (40.693113, -74.001909),
  (40.704957, -73.984639),
  (40.703770, -73.952255),
  (40.706509, -73.947252), # soccer field
  (40.636454, -73.846851),
]
sf_points = [
  (37.760997, -122.426817), # tennis
  (37.770148, -122.459013), # tennis
  (37.756706, -122.487113), # tennis
  (37.760790, -122.512281), # no tennis
  (37.786376, -122.504517), # no tennis
  (37.802297, -122.438926), # no tennis
]
"""

In [ ]:
def get_embeddings_many(
    latlons, start_date, end_date,
    size=256,
    model=None
):
    return [get_embeddings(lat, lon, start_date, end_date, size=size) for lat, lon in latlons]


In [ ]:
PATCH_SIZE = 32

embeddings_true, stack_true = zip(*get_embeddings_many(points_true, "2022-07-01", "2022-07-05", size=PATCH_SIZE))
embeddings_false, stack_false = zip(*get_embeddings_many(points_false, "2023-03-01", "2023-03-05", size=PATCH_SIZE))


In [ ]:
x = embeddings_true + embeddings_false
x = [i.flatten() for i in x]

y = [1] * len(embeddings_true) + [0] * len(embeddings_false)


In [ ]:
clf = svm.SVC()
clf.fit(x, y)

In [ ]:
embeddings_test, stack_test = zip(*get_embeddings_many(points_test, "2022-08-01", "2022-08-30", size=PATCH_SIZE))
z = [i.flatten() for i in embeddings_test]


In [ ]:
clf.predict(z)

In [ ]:
for stack in stack_false:
    stack.sel(band=["red", "green", "blue"]).plot.imshow(
        row="time", rgb="band", vmin=0, vmax=2000, col_wrap=6
    )